In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import matplotlib.pylab  as plt

In [2]:
tree = ET.parse('test.xml')
root = tree.getroot()

In [3]:
img = cv2.imread('test.jpg',cv2.IMREAD_COLOR)
rows,cols,ch = img.shape

## draw certain point

In [ ]:
for table in root.iter('table'):
    point = table[0].get('points').split()
    cell = []
    idx = 0
    for t in table.iter('cell'):
        if idx==3:   # point idx
            cell.append(t.find('Coords').get('points').split())
            break
        idx++

img_tmp = img
table_point = [(int(x[0]),int(x[1]))for x in [x.split(',') for x in point]]


cell_point = []
for c in cell:
    cell_point += c
cell_p = [(int(x[0]),int(x[1])) for x in [x.split(',') for x in cell_point]]

for p in table_point:
    cv2.circle(img_tmp,p, 4, (0,0,255), 4)
for c in cell_p:
    cv2.circle(img_tmp,c, 4, (0,0,255), 4)
    
height, width = img_tmp.shape[:2]
size = (int(width/4), int(height/4))  
img_shrink = cv2.resize(img_tmp, size, interpolation=cv2.INTER_AREA) 


cv2.imshow('image', img_shrink)
cv2.waitKey (10000) # 显示 10000 ms 即 10s 后消失

## draw points

In [37]:
img_tmp = img
table_point = [(int(x[0]),int(x[1]))for x in [x.split(',') for x in point]]


cell_point = []
for c in cell:
    cell_point += c
cell_p = [(int(x[0]),int(x[1])) for x in [x.split(',') for x in cell_point]]

for p in table_point:
    cv2.circle(img_tmp,p, 4, (0,0,255), 4)
for c in cell_p:
    cv2.circle(img_tmp,c, 4, (0,0,255), 4)
    
height, width = img_tmp.shape[:2]
size = (int(width/4), int(height/4))  
img_shrink = cv2.resize(img_tmp, size, interpolation=cv2.INTER_AREA) 


cv2.imshow('image', img_shrink)
cv2.waitKey (10000) # 显示 10000 ms 即 10s 后消失


255

## transform

In [59]:
xy = ([[int(x[0])/4,int(x[1])/4]for x in [x.split(',') for x in point]])

rows_t, cols_t = img_shrink.shape[:2]

pts1 = np.float32(xy)
pts2 = np.float32([[0,0],[rows_t,0],[rows_t, cols_t],[0,cols_t]])

M = cv2.getPerspectiveTransform(pts1,pts2)
dst = cv2.warpPerspective(img_shrink,M,(rows_t,cols_t))

cv2.imshow('image', dst)
cv2.waitKey (10000) # 显示 10000 ms 即 10s 后消失


255

## label

In [9]:
gray = np.zeros((300,300,1) ,dtype="uint8")
cv2.line(gray, (0, 0), (150, 150), (255,255,255),4)
cv2.imshow("gray", gray) 
cv2.waitKey(1000) 

255

In [8]:
def str2point(point):
    rows = len(point)
    res = np.zeros((rows,2),dtype='int32')
    
    for idx in range(rows):
        res[idx,0], res[idx,1] = point[idx].split(',')
    return res

def isSorted(l,isAcs = True):
    if isAcs:
        return all(l[i] <= l[i+1] for i in range(len(l)-1))
    else:
        return all(l[i] >= l[i+1] for i in range(len(l)-1))
def isAlign(l):
    return all(abs(l[i]-l[i+1])<=30 for i in range(len(l)-1))

def extractPoint(table):
    table_point = table.find('Coords').get('points').split()
    table_point = str2point(table_point)
    
    cell_point = []
    cell_loc = []
    for t in table.iter('cell'):
        tmp = t.find('Coords').get('points').split()
        cell_point.append(str2point(tmp))
        
        sr = int(t.get('start-row'))
        er = int(t.get('end-row'))
        sc = int(t.get('start-col'))
        ec = int(t.get('end-col'))
        cell_loc.append(np.array([sr,er,sc,ec]))
    
    return table_point, cell_point, cell_loc

def checkPoint(point, row, col):
    N = point.shape[0]
    flag = True
    if  N == 2*(row+col):
        vline = point[0:row+1,:]
        flag1 = isSorted(vline[:,1]) and isAlign(vline[:,0])
        hline = point[row:col+row+1,:]
        flag2 = isSorted(hline[:,0]) and isAlign(hline[:,1])
        vline = point[col+row:col+2*row+1,:]
        flag3 = isSorted(vline[:,1], False) and isAlign(vline[:,0])
        hline = point[col+2*row:N,:]
        flag4 = isSorted(hline[:,0], False) and isAlign(hline[:,1])
        if not(flag1 and flag2 and flag3 and flag4):
            flag = False
    else:
        flag = False
    return flag

In [9]:
def preprocess(img_path, xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    rows,cols,ch = img.shape

    img_table = []
    fm_table = []
    
    for table in root.iter('table'):
        table_point, cell_point ,cell_loc = extractPoint(table)

        #transformation
        pts1 = np.float32(table_point)
        pts2 = np.float32([[0,0],[rows,0],[rows, cols],[0,cols]])
        M = cv2.getPerspectiveTransform(pts1,pts2)

        #label
        horizon_map = np.zeros((rows,cols) ,dtype="uint8")
        vertical_map = np.zeros((rows,cols) ,dtype="uint8")

        assert len(cell_loc)==len(cell_point)

        for idx in range(len(cell_point)):
            cell_row = cell_loc[idx][1] - cell_loc[idx][0] + 1
            cell_col = cell_loc[idx][3] - cell_loc[idx][2] + 1
            N = cell_point[idx].shape[0]
            
            if checkPoint(cell_point[idx], cell_row, cell_col):          
                assert N==2*(cell_row+cell_col), cell_col
                # horizontal line feature map
                for i in range(cell_row, cell_row+cell_col):
                    cv2.line(horizon_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 4)           # visuable horizontal line
                for i in range(1, cell_row):
                    cv2.line(horizon_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][N-i-cell_col]), 2, 4)  # unvisuable horizontal line
                for i in range(cell_col+2*cell_row, N-1):
                    cv2.line(horizon_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 4)           # visuable horizontal line
                cv2.line(horizon_map, tuple(cell_point[idx][N-1]), tuple(cell_point[idx][0]), 1, 4)

                # vertical line feature map
                for i in range(cell_row):
                    cv2.line(vertical_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 4)           # visuable vertical line
                for i in range(cell_row+1, cell_col+cell_row):
                    cv2.line(vertical_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][N-i+cell_row]), 2, 4)  # unvisuable vertical line
                for i in range(cell_col+cell_row, cell_col+2*cell_row):
                    cv2.line(vertical_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 4)           # visuable vertical line
            else:
                print("%d\t%d\t%d\t%d" %(idx,N,cell_row,cell_col))
                idx1 = np.argmin(np.array([x[1]+x[0] for x in cell_point[idx]])) # down left
                idx2 = np.argmin(np.array([x[1]-x[0] for x in cell_point[idx]])) # down right
                idx3 = np.argmax(np.array([x[1]+x[0] for x in cell_point[idx]])) # top right
                cv2.line(vertical_map, tuple(cell_point[idx][0]), tuple(cell_point[idx][idx1]), 1, 4)
                cv2.line(horizon_map, tuple(cell_point[idx][idx1]), tuple(cell_point[idx][idx2]), 1, 4)
                cv2.line(vertical_map, tuple(cell_point[idx][idx2]), tuple(cell_point[idx][idx3]), 1, 4)
                cv2.line(horizon_map, tuple(cell_point[idx][idx3]), tuple(cell_point[idx][0]), 1, 4)

        horizon_map = cv2.warpPerspective(horizon_map,M,(rows,cols))
        vertical_map = cv2.warpPerspective(vertical_map,M,(rows,cols))
        fm_table.append((horizon_map,vertical_map))
        
        img_tmp = cv2.warpPerspective(img,M,(rows,cols))
        img_table.append(img_tmp)
    return img_table, fm_table

In [10]:
img, fm = preprocess('test.jpg', 'test.xml')

2	15	1	6
3	38	1	17
39	5	1	1
43	5	1	1
99	5	1	1
210	5	1	1
434	5	1	1
603	5	1	1


In [11]:
cv2.imshow("gray", fm[0][0]) 
cv2.waitKey(1000) 

255

In [7]:
for table in root.iter('table'):
        table_point, cell_point ,cell_loc = extractPoint(table)
visidx = 212
for table in root.iter('table'):
    point = table[0].get('points').split()
    cell = []
    idx = 0
    for t in table.iter('cell'):
        if idx==visidx:   # point idx
            cell.append(t.find('Coords').get('points').split())
            break
        idx+=1
        
print(cell)
print(len(cell[0]))
cell_row = cell_loc[visidx][1] - cell_loc[visidx][0] + 1
cell_col = cell_loc[visidx][3] - cell_loc[visidx][2] + 1
print(cell_row)
print(cell_col)

[['1787,1458', '1786,1514', '2233,1530', '2233,1472']]
4
1
1


In [56]:
cell_loc

[array([0, 5, 0, 1]),
 array([0, 5, 2, 2]),
 array([0, 0, 3, 8]),
 array([ 0,  0,  9, 25]),
 array([ 0,  5, 26, 27]),
 array([1, 1, 3, 3]),
 array([1, 1, 4, 8]),
 array([ 1,  1,  9, 10]),
 array([ 1,  1, 11, 12]),
 array([ 1,  1, 13, 25]),
 array([2, 4, 3, 3]),
 array([2, 4, 4, 4]),
 array([2, 2, 5, 7]),
 array([2, 4, 8, 8]),
 array([2, 5, 9, 9]),
 array([ 2,  5, 10, 10]),
 array([ 2,  5, 11, 11]),
 array([ 2,  5, 12, 12]),
 array([ 2,  5, 13, 13]),
 array([ 2,  5, 14, 14]),
 array([ 2,  5, 15, 15]),
 array([ 2,  5, 16, 16]),
 array([ 2,  5, 17, 17]),
 array([ 2,  4, 18, 18]),
 array([ 2,  5, 19, 19]),
 array([ 2,  5, 20, 20]),
 array([ 2,  5, 21, 21]),
 array([ 2,  5, 22, 22]),
 array([ 2,  5, 23, 23]),
 array([ 2,  5, 24, 24]),
 array([ 2,  5, 25, 25]),
 array([3, 3, 5, 6]),
 array([3, 3, 7, 7]),
 array([4, 4, 5, 7]),
 array([5, 5, 3, 3]),
 array([5, 5, 4, 4]),
 array([5, 5, 5, 7]),
 array([5, 5, 8, 8]),
 array([ 5,  5, 18, 18]),
 array([6, 6, 0, 0]),
 array([6, 7, 1, 1]),
 array([ 6